Script para obtener estadísticas de los corpus

In [1]:
# Cargamos las librerías necesarias
import numpy as np
import pandas as pd
import re
import os, glob
from sklearn.datasets import load_files

# Habilitamos display interactivo y enlace con nuestra cuenta de google
from google.colab import data_table
from google.colab import drive

In [2]:
# Enlazamos nuestro notebook en Colab con nuestro almacenamiento en Google Drive 
drive.mount('/content/drive')

Mounted at /content/drive


Cargamos el conjunto de entrenamiento leyendo todas las notas. Sin tokenizar

In [ ]:
DATA_DIR_TRAIN = "/content/drive/MyDrive/Colab Notebooks/Corpus/MEDDOPROF/Original/Train/"
data = load_files(DATA_DIR_TRAIN, encoding="utf-8", decode_error="replace", shuffle=False)
data_train = pd.DataFrame(data.data, columns = ['texto'])
data_train['fichero'] = data.filenames

# Nos quedamos únicamente con los ficheros de estudio en base a como se organiza el corpus en carpetas
data_train = data_train[(data_train.fichero.str.contains('.txt', case=False, regex=False)) & (data_train.fichero.str.contains('task1', case=False, regex=False))]

#df["deteccion"] = df['texto'].str.contains('trabaj|ocupacion|profesion', case=False, regex=True)

#Eliminamos la ruta para comparar el nombre de las notas a revisar con las presentes en el conjunto de test de MEDDOPROF
data_train['fichero'] = data_train["fichero"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
data_train.head(5)

Hacemos los propio con el conjunto de test

In [ ]:
DATA_DIR_TEST = "/content/drive/MyDrive/Colab Notebooks/Corpus/MEDDOPROF/Original/Test/"
data = load_files(DATA_DIR_TEST, encoding="utf-8", decode_error="replace", shuffle=False)

data_test = pd.DataFrame(data.data, columns = ['texto'])
data_test['fichero'] = data.filenames

# Nos quedamos únicamente con los ficheros de estudio en base a como se organiza el corpus en carpetas
data_test = data_test[(data_test.fichero.str.contains('.txt', case=False, regex=False)) & (data_test.fichero.str.contains('ner', case=False, regex=False))]

#df["deteccion"] = df['texto'].str.contains('trabaj|ocupacion|profesion', case=False, regex=True)

#Eliminamos la ruta para comparar el nombre de las notas a revisar con las presentes en el conjunto de test de MEDDOPROF
data_test['fichero'] = data_test["fichero"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
data_test.head(5)

Cogemos la versión tokenizada por spaCy del conjunto de train y test

In [ ]:
ruta_train = "/content/drive/MyDrive/Colab Notebooks/Corpus/MEDDOPROF/Procesado/NeuroNER/Train/task1neuronersm.txt"
ruta_test = "/content/drive/MyDrive/Colab Notebooks/Corpus/MEDDOPROF/Procesado/NeuroNER/Test/nerneuronersm.txt"

# Número de tokens

El número de tokens es igual al número de filas una vez hemos tokenizado con spaCy

In [ ]:
# Cargamos el archivo de train de la Tarea 1
# Si no ponemos on_bad_lines = "skip" obtenemos diferente valor en 876191	en	casos_clinicos_profesiones91	536	538	B-PROFESION
# Si no ponemos quoting = 3 obtenemos varios errores, por ejemplo, S0210-48062003000100010-1 917 918 O\nsecond S...	
train = pd.read_csv(ruta_train, sep=' ', 
                    names=['words', 'fileId', 'start', 'end', 'label'], quoting=3, on_bad_lines='skip')

# Transformamos las variables label y words
train['label'] = train['label'].astype('category')
train['words'] = train['words'].astype('str')

In [ ]:
# Cargamos el archivo de Test de la Tarea 1
test = pd.read_csv(ruta_test, sep=' ',
                   names=['words', 'fileId', 'start', 'end', 'label'], quoting=3, on_bad_lines='skip')

# Transformamos las variables label y words
test['label'] = test['label'].astype('category')
test['words'] = test['words'].astype('str')

### Número de tokens después de aplicar NEURONER

In [ ]:
# Número de tokens en el conjunto de train
len(train.words)

In [ ]:
# Número de tokens en el conjunto de test
len(test.words)

In [ ]:
len(train.words) + len(test.words)

# Número de sentences y caracteres

In [ ]:
train['sentence'] = train.groupby(['fileId'])['words'].transform(lambda x: ' '.join(x))
train['word_labels'] = train.groupby(['fileId'])['label'].transform(lambda x: ' '.join(x))

test['sentence'] = test.groupby(['fileId'])['words'].transform(lambda x: ' '.join(x))
test['word_labels'] = test.groupby(['fileId'])['label'].transform(lambda x: ' '.join(x))
train

In [ ]:
data_train

In [ ]:
# Recuento de frases
# https://stackoverflow.com/questions/63919115/count-how-many-sentences-there-are-in-each-row-within-a-pandas-column
!pip install textstat
import textstat

# Cuando aparecen número decimales falla (lo divide como una sentencia)
data_train['sentences'] = data_train['texto'].str.count('[\w][\.!\?]').clip(lower=1)
data_train['sentences_textstat'] = data_train['texto'].apply(textstat.sentence_count)
data_train['caracteres'] = data_train['texto'].str.len()
# Esto no se hace así, se hace con el tokenizador
# data_train["tokens"] = data_train.texto.str.count(' ')

# Tener en cuenta 
data_test['sentences'] = data_test['texto'].str.count('[\w][\.!\?]').clip(lower=1)
data_test['sentences_textstat'] = data_test['texto'].apply(textstat.sentence_count)
data_test['caracteres'] = data_test['texto'].str.len()
# Esto no se hace así, se hace con el tokenizador
# data_test["tokens"] = data_test.texto.str.count(' ')

# Recuento de tokens
# Importamos el tokenizador de spaCy
!pip install spacy
!python -m spacy download es_core_news_sm

import spacy
nlp = spacy.load("es_core_news_sm")
data_train['texto_tokenizado'] = data_train['texto'].apply(lambda x: nlp(x))
data_train["tokens"] = data_train.texto_tokenizado.agg([len])

data_test['texto_tokenizado'] = data_test['texto'].apply(lambda x: nlp(x))
data_test["tokens"] = data_train.texto_tokenizado.agg([len])


In [ ]:
# Vemos las discrepancias
data_train[(data_train.sentences != data_train.sentences_textstat) & (data_train.sentences < 10)]

In [ ]:
data_train

,texto,fichero,sentences,sentences_textstat,caracteres,texto_tokenizado,tokens
1,Un paciente de unos cincuenta años sin anteced...,32247016_ES,11,11,1612,"(Un, paciente, de, unos, cincuenta, años, sin,...",288
3,"﻿Una mujer de 27 años sin antecedentes, reside...",32294273_ES,8,8,838,"(﻿Una, mujer, de, 27, años, sin, antecedentes,...",140
5,﻿Mujer de 66 años que acudió refiriendo dificu...,32399950_ES,15,14,2765,"(﻿Mujer, de, 66, años, que, acudió, refiriendo...",450
7,﻿El 28 de febrero de 2020 ingresó en urgencias...,32417987_ES,20,20,2928,"(﻿El, 28, de, febrero, de, 2020, ingresó, en, ...",523
9,"﻿Una mujer de 31 años, por lo demás sana, acud...",32426200_ES,34,30,3927,"(﻿Una, mujer, de, 31, años, ,, por, lo, demás,...",664
...,...,...,...,...,...,...,...
2991,Se presenta el caso de una paciente de 28 años...,cc_otorrinolaringologia54,9,9,950,"(Se, presenta, el, caso, de, una, paciente, de...",157
2993,Paciente mujer de 81 años con hipoacusia bilat...,cc_otorrinolaringologia56,5,5,502,"(Paciente, mujer, de, 81, años, con, hipoacusi...",86
2995,Se presenta el caso de una mujer de 33 años qu...,cc_otorrinolaringologia60,9,9,1086,"(Se, presenta, el, caso, de, una, mujer, de, 3...",186
2997,Se presenta el caso de un varón de 18 años de ...,cc_otorrinolaringologia72,7,7,925,"(Se, presenta, el, caso, de, un, varón, de, 18...",166


Los tokens no se pueden calcular como el número de palabras entre espacios (eso no es tokenización, porque los caracteres de puntuación pueden formar tokens)

In [ ]:
# Número medio de sentence train
print("Sentences train:")
print("Número medio de sentences train: ", data_train.sentences.mean())
print("Número total de sentences train: ", data_train.sentences.sum())
print("Número mínimo de sentences train: ", data_train.sentences.min())
print("Número máximo de sentences train: ", data_train.sentences.max())
print(" ")
print("sentences_textstat train:")
print("Número medio de sentences_textstat train: ", data_train.sentences_textstat.mean())
print("Número total de sentences_textstat train: ", data_train.sentences_textstat.sum())
print("Número mínimo de sentences_textstat train: ", data_train.sentences_textstat.min())
print("Número máximo de sentences_textstat train: ", data_train.sentences_textstat.max())
print(" ")
print("Caracteres train:")
print("Número medio de caracteres train: ", data_train.caracteres.mean())
print("Número total de caracteres train: ", data_train.caracteres.sum())
print("Número mínimo de caracteres train: ", data_train.caracteres.min())
print("Número máximo de caracteres train: ", data_train.caracteres.max())
print(" ")

print("Tokens train:")
print("Número medio de tokens train: ", data_train.tokens.mean())
print("Número total de tokens train: ", data_train.tokens.sum())
print("Número mínimo de tokens train: ", data_train.tokens.min())
print("Número máximo de tokens train: ", data_train.tokens.max())
######
print(" ")
print("#############")
print(" ")
print("Sentences test:")
print("Número medio de sentences test: ", data_test.sentences.mean())
print("Número total de sentences test: ", data_test.sentences.sum())
print("Número mínimo de sentences test: ", data_test.sentences.min())
print("Número máximo de sentences test: ", data_test.sentences.max())
print(" ")
print("sentences_textstat train:")
print("Número medio de sentences_textstat test: ", data_test.sentences_textstat.mean())
print("Número total de sentences_textstat test: ", data_test.sentences_textstat.sum())
print("Número mínimo de sentences_textstat test: ", data_test.sentences_textstat.min())
print("Número máximo de sentences_textstat test: ", data_test.sentences_textstat.max())
print(" ")
print("Caracteres test:")
print("Número medio de caracteres test: ", data_test.caracteres.mean())
print("Número total de caracteres test: ", data_test.caracteres.sum())
print("Número mínimo de caracteres test: ", data_test.caracteres.min())
print("Número máximo de caracteres test: ", data_test.caracteres.max())
print(" ")
print("Tokens test:")
print("Número medio de tokens test: ", data_test.tokens.mean())
print("Número total de tokens test: ", data_test.tokens.sum())
print("Número mínimo de tokens test: ", data_test.tokens.min())
print("Número máximo de tokens test: ", data_test.tokens.max())

Sentences train:
Número medio de sentences train:  33.00933333333333
Número total de sentences train:  49514
Número mínimo de sentences train:  1
Número máximo de sentences train:  306
 
sentences_textstat train:
Número medio de sentences_textstat train:  33.071333333333335
Número total de sentences_textstat train:  49607
Número mínimo de sentences_textstat train:  1
Número máximo de sentences_textstat train:  247
 
Caracteres train:
Número medio de caracteres train:  4159.725333333334
Número total de caracteres train:  6239588
Número mínimo de caracteres train:  184
Número máximo de caracteres train:  27529
 
Tokens train:
Número medio de tokens train:  731.9333333333333
Número total de tokens train:  1097900
Número mínimo de tokens train:  29
Número máximo de tokens train:  4748
 
#############
 
Sentences test:
Número medio de sentences test:  27.863372093023255
Número total de sentences test:  9585
Número mínimo de sentences test:  1
Número máximo de sentences test:  220
 
sentence

In [ ]:
# Resultados de tokens totales
#1097900+240131
#1114919+222744

1337663

# Número de entidades

Vemos todas las etiquetas que hay

In [ ]:
train.label.value_counts()

O                      1067322
I-PROFESION               3698
B-PROFESION               2525
I-SITUACION_LABORAL       1531
B-SITUACION_LABORAL       1010
I-ACTIVIDAD                285
B-ACTIVIDAD                119
Name: label, dtype: int64

In [ ]:
# Train
# Nos quedamos con fila única
entidades_train = train[["fileId", "word_labels"]].drop_duplicates()
# Contamos cuantas veces aparecen las etiquetas en cada fila
entidades_train["nentidades"] = entidades_train.word_labels.str.count("B-PROFESION|I-PROFESION|B-SITUACION_LABORAL|I-SITUACION_LABORAL|B-ACTIVIDAD|I-ACTIVIDAD")

#Test
entidades_test = test[["fileId", "word_labels"]].drop_duplicates()
entidades_test["nentidades"] = entidades_test.word_labels.str.count("B-PROFESION|I-PROFESION|B-SITUACION_LABORAL|I-SITUACION_LABORAL|B-ACTIVIDAD|I-ACTIVIDAD")

In [ ]:
entidades_train

,fileId,word_labels,nentidades
0,32247016_ES,O O O O O O O O O O O O B-PROFESION I-PROFESIO...,14
287,32294273_ES,O O O O O O O O B-PROFESION I-PROFESION I-PROF...,4
426,32399950_ES,O O O O O O O O O O O O O O O O O O O O O O O ...,0
873,32417987_ES,O O O O O O O O O O B-PROFESION O O O O O O O ...,1
1395,32426200_ES,O O O O O O O O O O O O O O O O O O O O O O O ...,4
...,...,...,...
1075765,cc_otorrinolaringologia54,O O O O O O O O O O O O O O O O O O O O O O O ...,0
1075922,cc_otorrinolaringologia56,O O O O O O O O O O O O O O O O O O O O O O O ...,0
1076008,cc_otorrinolaringologia60,O O O O O O O O O O O O O O O O O O O O O O O ...,0
1076193,cc_otorrinolaringologia72,O O O O O O O O O O O O O B-PROFESION O O O O ...,1


In [ ]:
# Sacamos el resto de métricas
print("Número entidades totales train:", entidades_train.nentidades.sum())
print("Número entidades medias train:", entidades_train.nentidades.mean())
print("Número entidades medias train (de las frases con entidades):", entidades_train[entidades_train.nentidades >0].nentidades.mean())
print("Número entidades minimas train (de las frases con entidades):", entidades_train[entidades_train.nentidades >0].nentidades.min())
print("Número entidades maximas train (de las frases con entidades):", entidades_train[entidades_train.nentidades >0].nentidades.max())
print("")
print("Número entidades totales test:", entidades_test.nentidades.sum())
print("Número entidades medias test:", entidades_test.nentidades.mean())
print("Número entidades medias test (de las frases con entidades):", entidades_test[entidades_test.nentidades >0].nentidades.mean())
print("Número entidades minimas test (de las frases con entidades):", entidades_test[entidades_test.nentidades >0].nentidades.min())
print("Número entidades maximas test (de las frases con entidades):", entidades_test[entidades_test.nentidades >0].nentidades.max())

Número entidades totales train: 9168
Número entidades medias train: 6.112
Número entidades medias train (de las frases con entidades): 7.405492730210016
Número entidades minimas train (de las frases con entidades): 1
Número entidades maximas train (de las frases con entidades): 79

Número entidades totales test: 2778
Número entidades medias test: 8.075581395348838
Número entidades medias test (de las frases con entidades): 9.019480519480519
Número entidades minimas test (de las frases con entidades): 1
Número entidades maximas test (de las frases con entidades): 75


# Otras métricas

Cargamos cuatro archivos, 2 por cada tarea (ner / class) y 2 por cada conjunto (train / test)

Estos archivos los obtenemos haciendo lo siguiente:

1. Identificamos los segmentos del tipo: PROFESION 75 93 , con \t.*\t seleccionamos todo el texto, lo copiamos a otra hoja y sustituimos los espacios por tabulaciones
2. Una vez hecho esto seleccionamos los segmentos hasta el primer espacio “.* “: T1	PROFESION 75 93	médico de cabecera; con esto lo copiamos a otra hoja, cambiamos el espacio final lo cambiamos por tabulador y lo devolvemos a la hoja original

Sustituimos por \t

Para ver las instancias malas nos quedamos con el inicio y con el documento


`awk '{print $0,FILENAME}' *.ann >> ~/Desktop/profnervalid.txt`

EL OBJETIVO ES CONSEGUIR AGRUPAR TODOS LOS ARCHIVOS .ANN EN UN ÚNICO ARCHIVO .TXT

In [ ]:
task1train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Corpus/MEDDOPROF/Procesado/Anotaciones/task1_traincombinado_ann.txt", sep = "\t", names = ["Tag", "Entity", "start", "end", "freetext", "document"], header = None)
task1test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Corpus/MEDDOPROF/Procesado/Anotaciones/task1_testcombinado_ann.txt", sep = "\t", names = ["Tag", "Entity", "start", "end", "freetext", "document"], header = None)
task2train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Corpus/MEDDOPROF/Procesado/Anotaciones/task2_traincombinado_ann.txt", sep = "\t", names = ["Tag", "Entity", "start", "end", "freetext", "document"], header = None)
task2test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Corpus/MEDDOPROF/Procesado/Anotaciones/task2_testcombinado_ann.txt", sep = "\t", names = ["Tag", "Entity", "start", "end", "freetext", "document"], header = None)

In [ ]:
# Número de palabras
task1train["n_palabras"] = task1train.freetext.str.count(' ')
task1test["n_palabras"] = task1test.freetext.str.count(' ')
task2train["n_palabras"] = task2train.freetext.str.count(' ')
task2test["n_palabras"] = task2test.freetext.str.count(' ')

In [ ]:
task1train

,Tag,Entity,start,end,freetext,document,n_palabras
0,T1,PROFESION,75,93,médico de cabecera,32247016_ES.ann,2
1,T2,PROFESION,407,453,responsables de recursos de varios ministerios,32247016_ES.ann,5
2,T3,PROFESION,340,382,impartió varias conferencias sobre gestión,32247016_ES.ann,4
3,T1,PROFESION,271,282,dermatólogo,32294273_ES.ann,0
4,T2,PROFESION,40,62,residente del hospital,32294273_ES.ann,2
...,...,...,...,...,...,...,...
3653,T2,PROFESION,317,325,pediatra,cc_otorrinolaringologia32.ann,0
3654,T1,PROFESION,44,67,trabajador de la madera,cc_otorrinolaringologia371.ann,3
3655,T2,SITUACION_LABORAL,7730,7743,en el colegio,cc_otorrinolaringologia42.ann,2
3656,T1,PROFESION,18,39,trabajador del corcho,cc_otorrinolaringologia459.ann,2


In [ ]:
task1train.Entity.value_counts()

PROFESION            2528
SITUACION_LABORAL    1011
ACTIVIDAD             119
Name: Entity, dtype: int64

In [ ]:
task1test.Entity.value_counts()

PROFESION            699
SITUACION_LABORAL    358
ACTIVIDAD             28
Name: Entity, dtype: int64

In [ ]:
task2train.Entity.value_counts()

PACIENTE     1735
SANITARIO    1231
OTROS         485
FAMILIAR      207
Name: Entity, dtype: int64

In [ ]:
task2test.Entity.value_counts()

PACIENTE     592
SANITARIO    294
OTROS        146
FAMILIAR      53
Name: Entity, dtype: int64

In [ ]:
# Vemos distribución de etiquetas entre NER y CLASS del entrenamiento y test
task1_task2_train = pd.merge(task1train,task2train,on=["document", "start", "end", "freetext", "n_palabras"])
task1_task2_train

,Tag_x,Entity_x,start,end,freetext,document,n_palabras,Tag_y,Entity_y
0,T1,PROFESION,75,93,médico de cabecera,32247016_ES.ann,2,T1,SANITARIO
1,T2,PROFESION,407,453,responsables de recursos de varios ministerios,32247016_ES.ann,5,T2,OTROS
2,T3,PROFESION,340,382,impartió varias conferencias sobre gestión,32247016_ES.ann,4,T3,PACIENTE
3,T1,PROFESION,271,282,dermatólogo,32294273_ES.ann,0,T1,SANITARIO
4,T2,PROFESION,40,62,residente del hospital,32294273_ES.ann,2,T2,PACIENTE
...,...,...,...,...,...,...,...,...,...
3653,T2,PROFESION,317,325,pediatra,cc_otorrinolaringologia32.ann,0,T2,SANITARIO
3654,T1,PROFESION,44,67,trabajador de la madera,cc_otorrinolaringologia371.ann,3,T1,PACIENTE
3655,T2,SITUACION_LABORAL,7730,7743,en el colegio,cc_otorrinolaringologia42.ann,2,T2,PACIENTE
3656,T1,PROFESION,18,39,trabajador del corcho,cc_otorrinolaringologia459.ann,2,T1,PACIENTE


In [ ]:
task1_task2_train.value_counts(["Entity_x", "Entity_y"])

Entity_x           Entity_y 
PROFESION          SANITARIO    1231
                   PACIENTE      876
SITUACION_LABORAL  PACIENTE      754
PROFESION          OTROS         316
SITUACION_LABORAL  OTROS         160
ACTIVIDAD          PACIENTE      105
PROFESION          FAMILIAR      105
SITUACION_LABORAL  FAMILIAR       97
ACTIVIDAD          OTROS           9
                   FAMILIAR        5
dtype: int64

In [ ]:
task1_task2_test = pd.merge(task1test,task2test,on=["document", "start", "end", "freetext", "n_palabras"])
task1_task2_test.value_counts(["Entity_x", "Entity_y"])

Entity_x           Entity_y 
PROFESION          SANITARIO    294
SITUACION_LABORAL  PACIENTE     294
PROFESION          PACIENTE     282
                   OTROS         94
SITUACION_LABORAL  OTROS         43
PROFESION          FAMILIAR      29
SITUACION_LABORAL  FAMILIAR      23
ACTIVIDAD          PACIENTE      17
                   OTROS          9
                   FAMILIAR       2
dtype: int64

In [ ]:
# Encontramos los errores
comprobacion = task1_task2_test[["document", "start", "end"]]
comprobacion[comprobacion.duplicated(["document", "start", "end"], keep=False)]

,document,start,end
229,caso_clinico_psiquiatria304.ann,2686,2707
230,caso_clinico_psiquiatria304.ann,2686,2707
231,caso_clinico_psiquiatria304.ann,2686,2707
232,caso_clinico_psiquiatria304.ann,2686,2707
